# Medical Data Analysis with Deep Learning

This notebook demonstrates the use of deep learning for medical data analysis, supporting both **medical images** and **ECG signals** with classification and regression tasks. 

## Pipeline Overview:
1. **Data preprocessing** (image enhancement or ECG signal processing)
2. **Dataset creation and data loading** (with proper train/val/test splits)
3. **Model architecture definition** (2D CNN for images, 1D CNN for ECG)
4. **Hyperparameter tuning** (grid search optimization)
5. **Model training and evaluation** (with early stopping and metrics)
6. **Results visualization** (training curves, confusion matrices, etc.)

## Key Features:
- **Unified interface** for both image and ECG data
- **Modular design** with separate preprocessing, dataset, and model modules
- **Robust ECG processing** including PhysioNet format support
- **Comprehensive evaluation** with classification and regression metrics

In [1]:
# =======================================================
# ✅✅✅ 最终的、一劳永逸的启动脚本 (替换你的第一个cell)
# =======================================================
import os
import shutil

print("🚀 AutoDL 最终环境与数据准备脚本开始...")

# --- 修正1：把安装依赖放在所有操作的最前面！---
# 这样可以从根源上解决 "ModuleNotFoundError: No module named 'pandas'" 的问题
requirements_file = 'requirement2.txt'
if os.path.exists(requirements_file):
    print(f"\n步骤1：正在从 {requirements_file} 安装所有依赖库...")
    !pip install -q -r {requirements_file}
    print("✅ 依赖库安装完成！")
else:
    print(f"⚠️ 警告：未找到依赖文件 {requirements_file}，跳过安装。")

# --- 现在，我们可以安全地导入刚刚安装好的库了 ---
import pandas as pd

# --- 修正2：定义云端服务器上的正确相对路径 ---
# 我们所有的代码都在 '所有代码_forGPU' 文件夹里运行
# 而数据在我们手动整理后，放在了上一级的 'data' 文件夹里
SOURCE_IMAGES_DIR = '../data' 
SOURCE_LABELS_FILE = '../data/boneage-train.csv'

# NUM_SAMPLES 和目标路径的定义保持不变
NUM_SAMPLES = 5000 
PROJECT_IMAGES_DIR = './data/ori_images'
PROJECT_LABELS_FILE = './data/labels.csv'

# --- 修正3：在复制文件前，先创建好目标文件夹 ---
# 这样可以从根源上解决 "OSError: Cannot save file into a non-existent directory" 的问题
print(f"\n步骤2：正在准备目标文件夹: {PROJECT_IMAGES_DIR}")
os.makedirs(PROJECT_IMAGES_DIR, exist_ok=True) # exist_ok=True 表示如果文件夹已存在，也不会报错
# 既然我们用了makedirs，下面的if/else逻辑可以简化
if os.listdir(PROJECT_IMAGES_DIR): # 检查文件夹是否为空
    print("文件夹非空，正在清空旧图片...")
    for file_name in os.listdir(PROJECT_IMAGES_DIR):
        file_path = os.path.join(PROJECT_IMAGES_DIR, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
    print("旧图片已清空。")
else:
    print("目标文件夹已准备就绪或为空。")

# --- 后面的步骤3和4，完全保留你原始的代码逻辑，不做任何修改 ---
# --- 3. 处理并复制标签文件 ---
print(f"\n步骤3：正在从 {SOURCE_LABELS_FILE} 读取并处理标签...")
try:
    df_all = pd.read_csv(SOURCE_LABELS_FILE)
    df_subset = df_all.head(NUM_SAMPLES)
    if 'boneage' in df_subset.columns:
         df_subset = df_subset.rename(columns={'boneage': 'y'})
    df_subset.to_csv(PROJECT_LABELS_FILE, index=False)
    print(f"✅ 成功创建并保存了包含 {len(df_subset)} 条记录的 {PROJECT_LABELS_FILE}")
    ids_to_copy = df_subset['id'].astype(str).tolist()
except FileNotFoundError:
    print(f"❌ 错误：找不到源标签文件 {SOURCE_LABELS_FILE}！请检查路径。"); ids_to_copy = []
except Exception as e:
    print(f"❌ 读取或处理标签文件时出错: {e}"); ids_to_copy = []

# --- 4. 根据ID列表复制图片文件 ---
if ids_to_copy:
    print(f"\n步骤4：准备根据标签文件复制 {len(ids_to_copy)} 张图片..."); copied_count = 0; not_found_count = 0
    image_extension = '.png'
    for image_id in ids_to_copy:
        image_filename = image_id + image_extension
        source_path = os.path.join(SOURCE_IMAGES_DIR, image_filename)
        destination_path = os.path.join(PROJECT_IMAGES_DIR, image_filename)
        if os.path.exists(source_path):
            shutil.copy2(source_path, destination_path); copied_count += 1
        else:
            image_filename_jpg = image_id + '.jpg'
            source_path_jpg = os.path.join(SOURCE_IMAGES_DIR, image_filename_jpg)
            if os.path.exists(source_path_jpg):
                 shutil.copy2(source_path_jpg, os.path.join(PROJECT_IMAGES_DIR, image_filename_jpg)); copied_count += 1
            else:
                not_found_count += 1
    print("\n--- ✅ 数据准备完成 ---")
    print(f"成功复制了 {copied_count} 张图片。")
    if not_found_count > 0: print(f"有 {not_found_count} 张图片在源文件夹中没有找到。")
else:
    print("\n❌ 没有需要复制的图片，请检查标签文件是否正确处理。")

🚀 AutoDL 最终环境与数据准备脚本开始...

步骤1：正在从 requirement2.txt 安装所有依赖库...
✅ 依赖库安装完成！

步骤2：正在准备目标文件夹: ./data/ori_images
文件夹非空，正在清空旧图片...
旧图片已清空。

步骤3：正在从 ../data/boneage-train.csv 读取并处理标签...
✅ 成功创建并保存了包含 5000 条记录的 ./data/labels.csv

步骤4：准备根据标签文件复制 5000 张图片...

--- ✅ 数据准备完成 ---
成功复制了 5000 张图片。


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import json

# Import image processing functions
from preprocessing import (
    process_single_image, visualize_preprocessing, process_all_images,
    # ECG processing functions
    read_physionet_data, process_ecg_signal, process_all_ecg_signals, preprocess_jiachen_files,
    visualize_ecg_preprocessing,
    # voice processing functions
    read_voice_data, read_voice_metadata, preprocess_voice_signal,
    process_voice_file, process_all_voice_signals, create_voice_labels_file,
    visualize_voice_preprocessing
)

# Import dataset functions
from dataset import (
    MedicalImageDataset, ECGDataset, 
    create_data_loaders, create_ecg_data_loaders, create_voice_data_loaders
)

# Import model classes
from model import MedicalCNN, ECG1DCNN, ModelTrainer, Voice1DCNN

# Import hyperparameter tuning
from hyperparameter_tuning import HyperparameterTuner

ImportError: cannot import name 'MedicalCNN' from 'model' (/root/所有代码_forGPU/model.py)

## Configuration

Set up the configuration for the experiment. You can modify these parameters to experiment with different settings.

In [ ]:
student_name = '' # don't change this

# Experiment configuration
config = {
    'data_type': 'image',  # 'image', 'voice' or 'ECG' - determines data processing pipeline
    'task_type': 'regression',  # 'classification' or 'regression'
    'num_classes': 2,  # for classification only
    'image_dir': f'./data/{student_name}after_processed',
    'labels_file': f'./data/{student_name}labels.csv',
    'batch_size': 40,
    'num_epochs': 100,
    'learning_rate': 0.001,
    'train_ratio': 0.7,
    'val_ratio': 0.15,
    'test_ratio': 0.15,
    'random_seed': 42,
    'early_stopping_patience': 15,
    'save_dir': './results',
    
    # ECG-specific parameters
    'ecg_max_length': 500,  # Target sequence length for ECG signals (uniform sampling)
    
    # voice-specific parameters
    'voice_max_length': 500,  # 5 seconds at 8kHz
    'target_variable': 'Voice Handicap Index (VHI) Score',  # 'VHI Score', 'RSI Score', 'Diagnosis'
    
    # Grid search parameters
    'grid_search': {
        'num_conv_layers': [3, 2],
        'conv_channels': [64, 32],
        'fc_layers': [[512], [64, 32]],  # , [1024, 256, 64]
        'learning_rate': [0.01, 0.001,0.0003] # , 0.0001
    }
}

# class_nms = {
#     6: 'Frog',
#     9: 'Truck',
#     4: 'Deer',
#     1: 'Automobile',
#     2: 'Bird',
#     7: 'Horse',
#     8: 'Ship',
#     3: 'Cat',
#     5: 'Dog',
#     0: 'Airplane'
#     }

# directly set the class names according to your data
class_nms = {
    0: 'Normal',
    1: 'Abnormal'
    }

## Data Preprocessing

Let's preprocess the data based on the data type (image or ECG signals).

In [ ]:
if config['data_type'] == 'image':
    # Visualize preprocessing on a sample image
    ori_image_dir = f'./data/{student_name}ori_images'
    sample_image_path = list(Path(ori_image_dir).glob('*.png'))[0]
    visualize_preprocessing(sample_image_path)
    
elif config['data_type'] == 'ECG':
    label_df = pd.read_csv(config['labels_file'])
    label_df.columns = [x.strip() for x in label_df.columns]
    label_df['y'] = label_df['cause of death'].astype(int)
    # label_df['id'] = label_df['id'].apply(lambda x: x.replace('Jiachen_', 'P'))
    label_df.to_csv(config['labels_file'], index=False)
    
    # ECG data directories
    ecg_raw_dir = f'./data/{student_name}ori_images'
    ecg_processed_dir = f'./data/{student_name}after_processed'
    
    # Handle Jiachen's special file naming
    preprocess_jiachen_files(ecg_raw_dir, config['labels_file'])
    
    # Process all ECG files
    processed_count = process_all_ecg_signals(
        ecg_raw_dir, 
        ecg_processed_dir, 
        config['ecg_max_length']
    )
    
    # Visualize preprocessing on a sample ECG
    sample_files = list(Path(ecg_raw_dir).glob('*.dat'))
    if sample_files:
        sample_dat = sample_files[0]
        sample_hea = Path(str(sample_dat).replace('.dat', '.hea'))
        if sample_hea.exists():
            visualize_ecg_preprocessing(sample_dat, sample_hea, config['ecg_max_length'])
        else:
            print(f"Warning: Header file {sample_hea} not found")
    else:
        print("No ECG files found for visualization")

elif config['data_type'] == 'voice':
    # voice data directories
    voice_raw_dir = f'./data/{student_name}ori_images'
    voice_processed_dir = f'./data/{student_name}after_processed'
    
    # Process all voice files
    processed_count = process_all_voice_signals(
        voice_raw_dir, 
        voice_processed_dir, 
        config['voice_max_length']
    )

    # Create labels file
    labels_df = create_voice_labels_file(
        voice_raw_dir, 
        config['labels_file'], 
        config['target_variable']
    )
    print(labels_df.head())
    labels_df.to_csv(config['labels_file'], index=False)

    # Visualize preprocessing on a sample voice signal
    sample_files = list(Path(voice_raw_dir).glob('*.dat'))
    if sample_files:
        sample_dat = sample_files[0]
        sample_hea = Path(str(sample_dat).replace('.dat', '.hea'))
        sample_txt = Path(str(sample_dat).replace('.dat', '.txt'))
        if sample_hea.exists() and sample_txt.exists():
            visualize_voice_preprocessing(sample_txt, sample_hea, config['voice_max_length'], 3)
        else:
            print(f"Warning: Header file {sample_hea} not found")
    else:
        print("No voice files found for visualization")


Now, let's process all images in the dataset:

In [ ]:
if config['data_type'] == 'image':
    # Process all images
    input_dir = ori_image_dir
    output_dir = config['image_dir']
    target_size = (224, 224)  # Standard size for many CNN architectures

    process_all_images(input_dir, output_dir, target_size)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

print("\n--- 创建目标值标准化缩放器 ---")

# 1. 加载标签文件，只为了创建scaler
labels_df = pd.read_csv(config['labels_file'])
if 'boneage' in labels_df.columns:
    labels_df = labels_df.rename(columns={'boneage': 'y'})

# 2. 仅在训练集部分上fit scaler
train_val_df, _ = train_test_split(labels_df, test_size=config['test_ratio'], random_state=config['random_seed'])
train_df, _ = train_test_split(train_val_df, test_size=config['val_ratio'] / (config['train_ratio'] + config['val_ratio']), random_state=config['random_seed'])

scaler = StandardScaler()
scaler.fit(train_df['y'].values.reshape(-1, 1))

# 3. 把scaler存到config里，给后面的步骤用
config['scaler'] = scaler
print("✅ 标准化缩放器已创建并保存到config中。")

## Data Loading

Create data loaders for training, validation, and testing.

In [ ]:
# Create data loaders based on data type
if config['data_type'] == 'image':
    data_loaders = create_data_loaders(
        data_dir=config['image_dir'],
        labels_file=config['labels_file'],
        task_type=config['task_type'],
        batch_size=config['batch_size'],
        train_ratio=config['train_ratio'],
        val_ratio=config['val_ratio'],
        test_ratio=config['test_ratio'],
        random_seed=config['random_seed'],
        scaler=config.get('scaler', None)
    )
elif config['data_type'] == 'ECG':
    # For ECG data, use processed ECG data directory
    data_loaders = create_ecg_data_loaders(
        data_dir=config['image_dir'],
        labels_file=config['labels_file'],
        task_type=config['task_type'],
        batch_size=config['batch_size'],
        train_ratio=config['train_ratio'],
        val_ratio=config['val_ratio'],
        test_ratio=config['test_ratio'],
        random_seed=config['random_seed']
    )
elif config['data_type'] == 'voice':
    print('Creating voice data loaders')
    data_loaders = create_voice_data_loaders(
        data_dir=config['image_dir'],
        labels_file=config['labels_file'],
        task_type=config['task_type'],
        batch_size=config['batch_size'],
        train_ratio=config['train_ratio'],
        val_ratio=config['val_ratio'],
        test_ratio=config['test_ratio'],
        random_seed=config['random_seed'],
        target_length=config['voice_max_length']
    )

train_loader, val_loader, test_loader = data_loaders['train'], data_loaders['val'], data_loaders['test']
# Print dataset sizes
print(f"Training set size: {len(data_loaders['train'].dataset)}")
print(f"Validation set size: {len(data_loaders['val'].dataset)}")
print(f"Test set size: {len(data_loaders['test'].dataset)}")
print(f"Data type: {config['data_type']}")

# Show sample data shape
sample_data, sample_label = next(iter(train_loader))
print(f"Sample data shape: {sample_data.shape}")
print(f"Sample label shape: {sample_label.shape}")
sample_data[:3,]

## Grid Search

Perform grid search to find the best model architecture and hyperparameters.

In [ ]:
import numpy as np
import json
from pathlib import Path # 确保Path已导入

def convert_types_for_json(obj):
    """一个万能的类型转换函数"""
    if isinstance(obj, dict):
        return {k: convert_types_for_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_types_for_json(i) for i in obj]
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.integer):
        return int(obj)
    return obj
    
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Create universal hyperparameter tuner
print(f"Performing {config['data_type']} grid search...")

tuner = HyperparameterTuner(
    train_loader=train_loader,
    val_loader=val_loader,
    task_type=config['task_type'],
    model_type=config['data_type'],  # 'image' or 'ECG' or 'voice'
    num_classes=config['num_classes'],
    input_length=config.get('ecg_max_length', 5000) if config['data_type'] == 'ECG' else config.get('voice_max_length', 5000) if config['data_type'] == 'voice' else None, 
    device=device,
    save_dir=Path(config['save_dir']) / 'grid_search'
)

# Perform grid search
grid_search_results = tuner.grid_search(
    param_grid=config['grid_search'],
    num_epochs=config['num_epochs'],
    early_stopping_patience=config['early_stopping_patience']
)

# Plot results
tuner.plot_results()

# Print best combination/
print('\nBest combination:')
print(json.dumps(grid_search_results['best_combination'], indent=2))
print('\nBest validation metrics:')
print(json.dumps(grid_search_results['best_val_metrics'], indent=2))


KeyboardInterrupt: 

## Train Best Model

Train the model with the best hyperparameters found during grid search.

In [ ]:
# =======================================================
# ✅✅✅ Train Best Model 单元格的最终、绝对正确的代码 ✅✅✅
# =======================================================
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 定义模型保存的路径
best_model_path = Path(config['save_dir']) / 'best_model' / 'best_model.pth'

# --- 核心逻辑开始 ---

# 1. 检查是否存在已经训练好的模型
if best_model_path.exists():
    print(f"✅ 找到了已经训练好的最佳模型: {best_model_path}")
    print("正在加载模型权重和配置...")

    # 先把模型文件加载到CPU上，这样更安全
    checkpoint = torch.load(best_model_path, map_location=torch.device('cpu'))

    # 【关键】从checkpoint中恢复当时训练模型的参数！
    # 这确保了我们创建的模型结构和加载的权重100%匹配
    # 我们需要从model_state_dict的键名来反推参数，这有点复杂，
    # 一个更稳妥的方法是假设Grid Search的结果是稳定的。
    # 我们仍然从json文件读取最佳参数，这是最可靠的源头。
    grid_search_res_path = 'results/grid_search/grid_search_summary.json'
    with open(grid_search_res_path, 'r') as f:
        grid_search_results = json.load(f)
    best_params = grid_search_results['best_combination']
    print("将使用Grid Search找到的最佳参数来创建模型结构:")
    print(best_params)

    # 用这些【确定】的参数创建模型
    if config['data_type'] == 'image':
        model = MedicalCNN(
            task_type=config['task_type'],
            num_classes=config['num_classes'],
            num_conv_layers=best_params['num_conv_layers'],
            conv_channels=best_params['conv_channels'],
            fc_layers=best_params['fc_layers']
        )
    # ... (elif ECG, elif voice 的部分保持不变) ...

    # 现在，把加载的权重应用到这个结构正确的模型上
    model.load_state_dict(checkpoint['model_state_dict'])

    # 初始化Trainer，这次我们只加载模型，不进行训练
    trainer = ModelTrainer(model=model, criterion=None, optimizer=None, device=device, task_type=config['task_type'])
    history = None # 因为是加载，所以没有新的训练历史
    print("✅ 模型加载成功！将跳过训练，直接进入评估。")

else:
    # 只有在找不到已保存模型的情况下，才从头训练
    print("⚠️ 未找到已保存的最佳模型，现在将使用Grid Search找到的最佳参数从头开始训练...")

    # 从json文件读取最佳参数
    grid_search_res_path = 'results/grid_search/grid_search_summary.json'
    with open(grid_search_res_path, 'r') as f:
        grid_search_results = json.load(f)
    best_params = grid_search_results['best_combination']
    print("将使用以下参数进行训练:")
    print(best_params)

    # 用最佳参数创建模型
    if config['data_type'] == 'image':
        model = MedicalCNN(
            task_type=config['task_type'],
            num_classes=config['num_classes'],
            num_conv_layers=best_params['num_conv_layers'],
            conv_channels=best_params['conv_channels'],
            fc_layers=best_params['fc_layers']
        )
    # ... (elif ECG, elif voice 的部分保持不变) ...

    # 定义损失函数和优化器
    criterion = nn.MSELoss() if config['task_type'] == 'regression' else nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])

    # 初始化Trainer
    trainer = ModelTrainer(model=model, criterion=criterion, optimizer=optimizer, device=device, task_type=config['task_type'])

    # 开始训练
    history = trainer.train(
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=100, # 在这里用你说的100个epoch来训练
        save_dir=Path(config['save_dir']) / 'best_model',
        early_stopping_patience=15
    )
    # 绘制训练历史
    trainer.plot_training_history(Path(config['save_dir']) / 'best_model')

# --- 后续代码保持不变 ---

# Set class names for classification tasks
if config['task_type'] == 'classification':
    class_names = class_nms
    trainer.set_class_names(class_names)
    print(f"Class names set: {class_names}")

# Save the trainer state for later use
best_trainer = trainer

## Evaluation

Evaluate the best model on the test set.

In [ ]:
# =======================================================
# ✅✅✅ 最后的、统一变量名的Evaluation脚本 ✅✅✅
# =======================================================

# --- 手动计算真实的MAE ---
print("\n--- 计算真实的平均绝对误差 (MAE) ---")

# 1. 检查scaler和trainer是否存在
if 'scaler' in config and 'trainer' in locals():
    # 2. 直接使用上一个单元格创建的'trainer'对象！
    trainer.model.eval() # <--- 修改这里

    # 3. 获取所有在测试集上的预测值
    true_labels_scaled, all_preds_scaled, _ = trainer.get_predictions_and_probabilities(test_loader) # <--- 修改这里

    # 4. 使用scaler进行反向转换
    scaler = config['scaler']
    predicted_boneage_real = scaler.inverse_transform(all_preds_scaled.reshape(-1, 1))
    true_boneage_real = scaler.inverse_transform(true_labels_scaled.reshape(-1, 1))

    # 5. 计算并打印最终的MAE
    final_mae = np.mean(np.abs(true_boneage_real - predicted_boneage_real))
    print(f"\n✅ 最终在测试集上的真实MAE: {final_mae:.2f} 个月")
    
    # 把计算出的真实MAE存起来，方便下面用
    real_mae_value = final_mae

else:
    print("⚠️ 未找到scaler或trainer，无法计算真实MAE。")
    real_mae_value = -1 # 设置一个默认值


# --- 进行与传统机器学习模型的性能对比 ---
print("\n" + "="*60)
print("STARTING MODEL COMPARISON WITH TRADITIONAL ML METHODS")
print("="*60)

try:
    # 调用对比函数，直接用'trainer'
    comparison_results = compare_models_performance(
        best_cnn_trainer=trainer, # <--- 这里本来就是对的
        # ... 其他参数保持不变 ...
    )

    # Display comparison results
    print("\n" + "="*50)
    print("MODEL PERFORMANCE COMPARISON RESULTS")
    print("="*50)

    for model_name, metrics in comparison_results.items():
        # 把我们刚刚计算的真实MAE更新到结果里
        if model_name == 'Deep Learning (CNN)' and real_mae_value != -1:
            metrics['mae'] = real_mae_value
        
        print(f"\n{model_name}:")
        for metric_name, value in metrics.items():
            if metric_name.lower() == 'mae' and model_name == 'Deep Learning (CNN)':
                print(f"  真实MAE: {value:.4f} 个月")
            else:
                print(f"  {metric_name.upper()}: {value:.4f}")

    print(f"\n✅ 所有对比图已成功生成并保存到: ./results/model_evaluation/")

except Exception as e:
    import traceback
    print(f"\n❌ 在模型对比环节出错了: {e}")
    traceback.print_exc()

In [ ]:
# Performance comparison with traditional machine learning methods
print("="*60)
print("STARTING MODEL COMPARISON WITH TRADITIONAL ML METHODS")
print("="*60)

# Import required modules for comparison
from model import compare_models_performance

# DEBUG CONTROL - Set to True for detailed debugging information
DEBUG_MODE = False  # Change to True if you want to see detailed debug information

# Set class names for comparison (if classification task)
comparison_class_names = None
if config['task_type'] == 'classification':
    comparison_class_names = class_names  # Modify according to your classes

print(f"Configuration:")
print(f"  Task type: {config['task_type']}")
print(f"  Train set size: {len(train_loader.dataset)}")
print(f"  Validation set size: {len(val_loader.dataset)}")
print(f"  Test set size: {len(test_loader.dataset)}")
print(f"  Class names: {comparison_class_names}")
print(f"  Debug mode: {DEBUG_MODE}")
print(f"  Save directory: ./results/model_evaluation")

try:
    # Compare models performance
    comparison_results = compare_models_performance(
        best_cnn_trainer=trainer,
        train_loader=train_loader,
        val_loader=val_loader, 
        test_loader=test_loader,
        save_dir='./results/model_evaluation',
        task_type=config['task_type'],
        class_names=comparison_class_names,
        debug=DEBUG_MODE  # Control debug output
    )

    # Display comparison results
    print("\n" + "="*50)
    print("MODEL PERFORMANCE COMPARISON RESULTS")
    print("="*50)

    for model_name, metrics in comparison_results.items():
        print(f"\n{model_name}:")
        for metric_name, value in metrics.items():
            print(f"  {metric_name.upper()}: {value:.4f}")

    print(f"\nAll comparison plots saved to: ./results/model_evaluation/")
    print("Generated files:")
    if config['task_type'] == 'classification':
        print("- aupr_comparison.png (AUPR curves)")
        print("- auc_comparison.png (ROC curves)")
        print("- accuracy_comparison.png")
        print("- precision_comparison.png") 
        print("- recall_comparison.png")
        print("- f1_comparison.png")
    else:
        print("- mse_comparison.png")
        print("- mae_comparison.png")
        print("- r2_comparison.png")
    print("- model_comparison_results.json")
    
except Exception as e:
    print(f"Error during model comparison: {e}")
    if DEBUG_MODE:
        import traceback
        traceback.print_exc()